# Deep Learning based FACs and Action Unit occurrence and Intensity Estimation

#### \- Amogh Gudi, Emrah Tasli, Tim den Uyl, Andreas Marulis
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7284873

### Plan
1. Load dataset and labels
2. Build a model
3. Train Model
4. Test Model

**Importing Libraries**

In [1]:
import keras
import numpy as np

/home/amogh/anaconda3/envs/faus_dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Loading dataset and labels

a. Loading dataset

In [5]:
BP4D_base_path='/home/amogh/cmu/dataset/BP4D/'

In [6]:
BP4D_training_folder=BP4D_base_path+'BP4D-training/'

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
gen=ImageDataGenerator(horizontal_flip=True)

Batches from directory:

In [9]:
batch_size=32
batches_from_dir=gen.flow_from_directory(BP4D_training_folder,target_size=(48,48),batch_size=batch_size)

Found 553 images belonging to 1 classes.


b. Loading labels

In [10]:
BP4D_AU_path=BP4D_base_path+'AUCoding/AUCoding/'

Function to get labels according to subject, sequence, frame No: (details and examples in datasets.ipynb)

In [11]:
def label_getter(subj_req,seq_req,frame_req):
    au_file_reqd=BP4D_AU_path+f'{subj_req}_{seq_req}.csv'
#     print (au_file_reqd)
    df_reqd=pd.read_csv(au_file_reqd)
    list_of_faus=list(df_reqd.loc[df_reqd['0']==frame_req].iloc[0]) #list: [frame_no, fau1, fau2, fau3....]
    #choose FAUS for which you want the labels.
    list_mask=[1,2,4,6,7,10,12,14,15,17,23]
    list_final=[list_of_faus[i] for i in list_mask]
    return list_final

In [12]:
label_getter('F001','T1',2440)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

We define our own flow_from_directory function so that we can assign multiple labels to the training data before passing it to the training function

In [13]:
file_names=batches_from_dir.filenames
file_names

['F001/T1/2440.jpg',
 'F001/T1/2441.jpg',
 'F001/T1/2442.jpg',
 'F001/T1/2443.jpg',
 'F001/T1/2444.jpg',
 'F001/T1/2445.jpg',
 'F001/T1/2446.jpg',
 'F001/T1/2447.jpg',
 'F001/T1/2448.jpg',
 'F001/T1/2449.jpg',
 'F001/T1/2450.jpg',
 'F001/T1/2451.jpg',
 'F001/T1/2452.jpg',
 'F001/T1/2453.jpg',
 'F001/T1/2454.jpg',
 'F001/T1/2455.jpg',
 'F001/T1/2456.jpg',
 'F001/T1/2457.jpg',
 'F001/T1/2458.jpg',
 'F001/T1/2459.jpg',
 'F001/T1/2460.jpg',
 'F001/T1/2461.jpg',
 'F001/T1/2462.jpg',
 'F001/T1/2463.jpg',
 'F001/T1/2464.jpg',
 'F001/T1/2465.jpg',
 'F001/T1/2466.jpg',
 'F001/T1/2467.jpg',
 'F001/T1/2468.jpg',
 'F001/T1/2469.jpg',
 'F001/T1/2470.jpg',
 'F001/T1/2471.jpg',
 'F001/T1/2472.jpg',
 'F001/T1/2473.jpg',
 'F001/T1/2474.jpg',
 'F001/T1/2475.jpg',
 'F001/T1/2476.jpg',
 'F001/T1/2477.jpg',
 'F001/T1/2478.jpg',
 'F001/T1/2479.jpg',
 'F001/T1/2480.jpg',
 'F001/T1/2481.jpg',
 'F001/T1/2482.jpg',
 'F001/T1/2483.jpg',
 'F001/T1/2484.jpg',
 'F001/T1/2485.jpg',
 'F001/T1/2486.jpg',
 'F001/T1/248

In [14]:
def multiclass_flow_from_directory(batches_from_dir, label_getter):
    batch_count=0
    for x, y in batches_from_dir:
#         print ('batch number is: ',batch_count, ' file indexes from ', batch_size*batch_count,' and ', batch_size*batch_count+batch_size-1)
        first_image_index=batch_size*batch_count
        last_image_index=batch_size*batch_count+batch_size-1
        fau_labels=np.array([label_getter(file.split('.')[0].split('/')[0],file.split('.')[0].split('/')[1],int(file.split('.')[0].split('/')[2])) for file in file_names[first_image_index:last_image_index+1]])
        batch_count+=1
        print('x:    ',type(x),x.shape)
        print('fau_labels:    ',type(fau_labels),fau_labels.shape)
        yield x, fau_labels

In [15]:
generator_with_modified_labels=multiclass_flow_from_directory(batches_from_dir,label_getter)

In [16]:
keras.utils.to_categorical((3,4,4,1),6)

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]])

### 2. Building a Model

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D,Flatten,Dropout, Lambda
from keras.optimizers import SGD

In [18]:
model=Sequential()
model.add(Lambda(lambda x: tf.image.rgb_to_grayscale(x),input_shape=(48,48,3)))
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu'))
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu'))
model.add(Conv2D(128,kernel_size=(4,4),strides=(1,1),activation='relu'))
model.add(Flatten())
model.add(Dense(3072,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(11,activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 64)        102464    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       131200    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3072)              77073408  
__________

### 3. Training the model

In [21]:
model.fit_generator(generator_with_modified_labels,steps_per_epoch=20)

Epoch 1/1
x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 1/20 [>.............................] - ETA: 1:27 - loss: 2.1366x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 2/20 [==>...........................] - ETA: 56s - loss: 1.1381 x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 3/20 [===>..........................] - ETA: 51s - loss: 0.7587x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 4/20 [=====>........................] - ETA: 48s - loss: 0.8974x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 5/20 [======>.......................] - ETA: 44s - loss: 1.2322x:     <class 'numpy.ndarray'> (32, 48, 48, 3)
fau_labels:     <class 'numpy.ndarray'> (32, 11)
 6/20 [========>.....................] - ETA: 41s - loss: 1.8076x:     <class 'numpy.ndarray

KeyboardInterrupt: 

### 4. Testing the model